# Integer Division & Modulus — Practice Notebook

Hands-on exercises for `//` (floor division), `%` (modulus), and `divmod()`.

Key identity (Euclidean division): for integers `a` (dividend) and nonzero `b` (divisor),

```
q = a // b   # floor quotient
r = a % b    # remainder
a == q*b + r
```

- In Python, `//` is **floor** division (rounds toward `-∞`).
- The remainder `r` has the **same sign as the divisor** `b` (and is `0` when divisible).
- `divmod(a, b)` returns `(q, r)` efficiently.


## 1) Warm-up: basic identities with positives
Verify the fundamental identity and equivalence with `divmod`.


In [1]:
def check_identity(a, b):
    q, r = divmod(a, b)
    assert a == q*b + r
    return q, r

pairs = [(10, 3), (27, 5), (100, 10), (623, 60)]
for a, b in pairs:
    q, r = check_identity(a, b)
    print(f"a={a}, b={b} -> q={q}, r={r}")
print("OK ✅")


a=10, b=3 -> q=3, r=1
a=27, b=5 -> q=5, r=2
a=100, b=10 -> q=10, r=0
a=623, b=60 -> q=10, r=23
OK ✅


## 2) `divmod(a, b)` vs `a // b` and `a % b`
They’re equivalent, but `divmod` computes both at once (and is often clearer).


In [2]:
a, b = 165, 60
q1, r1 = a // b, a % b
q2, r2 = divmod(a, b)
assert (q1, r1) == (q2, r2)
print("// ->", q1, " % ->", r1, " | divmod ->", (q2, r2))


// -> 2  % -> 45  | divmod -> (2, 45)


## 3) Negative numbers: floor division rounds **down**
Python’s `//` rounds toward `-∞`, so be careful:

- `-7 // 3 == -3` (since `-2.333…` floored is `-3`)
- `-7 % 3 == 2` (so that `-7 == 3*(-3) + 2`)
- Remainder sign follows the divisor (`b`).


In [3]:
# a negative, b positive
assert (-7) // 3 == -3
assert (-7) % 3 == 2
assert (-7) == 3*((-7)//3) + ((-7)%3)

# a positive, b negative
assert 7 // (-3) == -3
assert 7 % (-3) == -2
assert 7 == (-3)*(7//-3) + (7%-3)

# both negative
assert (-7) // (-3) == 2
assert (-7) % (-3) == -1
assert (-7) == (-3)*((-7)//-3) + ((-7)%-3)

print("All negative-cases OK ✅")


All negative-cases OK ✅


## 4) Time conversion with `divmod`
Split total minutes into `(hours, minutes)` with a single call.


In [4]:
def split_h_m(elapsed_minutes: int):
    return divmod(elapsed_minutes, 60)

assert split_h_m(165) == (2, 45)
assert split_h_m(623) == (10, 23)
print("OK ✅", split_h_m(165), split_h_m(623))


OK ✅ (2, 45) (10, 23)


## 5) Even/odd and multiples
Use `%` to test parity and divisibility.


In [5]:
def is_even(n: int) -> bool:
    return n % 2 == 0

def is_multiple_of(n: int, k: int) -> bool:
    return n % k == 0

assert is_even(10) and not is_even(11)
assert is_multiple_of(21, 7) and not is_multiple_of(22, 7)
print("OK ✅")


OK ✅


## 6) Weekday arithmetic (clock math / modular arithmetic)
Let Monday = 0, …, Sunday = 6. What day is it `n` days later?


In [6]:
DAYS = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
def add_days(start_idx: int, delta: int) -> int:
    return (start_idx + delta) % 7

assert DAYS[add_days(0, 3)] == "Thu"   # Mon + 3 -> Thu
assert DAYS[add_days(5, 10)] == "Tue"  # Sat + 10 -> Tue
assert DAYS[add_days(2, -3)] == "Sun"  # Wed - 3 -> Sun
print("OK ✅")


OK ✅


## 7) Cycling through a list with `%`
Index safely with `% len(seq)` to loop around.


In [7]:
colors = ["red","green","blue"]
def cycle_get(seq, i):
    return seq[i % len(seq)]

assert cycle_get(colors, 0) == "red"
assert cycle_get(colors, 3) == "red"
assert cycle_get(colors, -1) == "blue"
print("OK ✅")


OK ✅


## 8) Pagination (pages & leftover)
Given `n_items` and `per_page`, compute `(n_pages, last_page_items)`.
- `n_pages = ceil(n_items / per_page)` → via `//` and `%` without `math.ceil`.
- `last_page_items = n_items % per_page` (or `per_page` when it divides exactly).


In [8]:
def paginate(n_items: int, per_page: int):
    q, r = divmod(n_items, per_page)
    n_pages = q if r == 0 else q + 1
    last_items = r if r != 0 else (per_page if n_items else 0)
    return n_pages, last_items

assert paginate(0, 10) == (0, 0)
assert paginate(100, 10) == (10, 10)
assert paginate(101, 10) == (11, 1)
assert paginate(237, 25) == (10, 12)
print("OK ✅")


OK ✅


## 9) GCD via Euclid’s algorithm (uses `%`)
Classic algorithm: repeatedly replace `(a, b)` with `(b, a % b)` until `b == 0`.


In [9]:
def gcd(a: int, b: int) -> int:
    a, b = abs(a), abs(b)
    while b:
        a, b = b, a % b
    return a

assert gcd(54, 24) == 6
assert gcd(48, 180) == 12
assert gcd(-42, 56) == 14
print("OK ✅")


OK ✅


## 10) Base conversion by repeated division and modulus
Convert a nonnegative integer to a base (2..36) string using `//` and `%`.


In [10]:
DIGITS = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def to_base(n: int, base: int = 2) -> str:
    if n == 0:
        return "0"
    if not (2 <= base <= 36):
        raise ValueError("base must be in 2..36")
    out = []
    while n > 0:
        n, r = divmod(n, base)
        out.append(DIGITS[r])
    return "".join(reversed(out))

assert to_base(13, 2) == "1101"
assert to_base(255, 16) == "FF"
assert to_base(12345, 36) == "9IX"
print("OK ✅")


OK ✅


## 11) Quick quiz — predict before you run
Compute mentally, then verify with asserts.


In [11]:
# A) identities
a, b = 27, 5
q, r = a // b, a % b
assert a == q*b + r and (q, r) == divmod(a, b)

# B) negatives (remember: floor division)
assert (-10) // 3 == -4
assert (-10) % 3 == 2
assert 10 // (-3) == -4
assert 10 % (-3) == -2

# C) time split
h, m = divmod(1000, 60)
assert (h, m) == (16, 40)

# D) parity & multiples
assert all((n % 2 == 0) for n in (0, 2, 4, 100))
assert (45 % 9 == 0) and (46 % 9 != 0)

# E) cycling index
seq = [9, 8, 7]
assert seq[(5) % len(seq)] == 7
assert seq[(-1) % len(seq)] == 7

print("All quiz checks passed ✅")


All quiz checks passed ✅
